In [ ]:
import os
import pandas as pd
import numpy as np
import copy
import sys
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 50)
sys.path.append("/Users/peterpriestley/hmf/repos/scripts/analysisscripts") 
import chromosomeDefinition as cd
import analyseVCF as aVCF
from scipy.stats import norm
from sklearn.neighbors import KernelDensity

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
CNV_COLUMNS = ['chrom','posStart','posEnd','copyNum','gainOrLoss','BAF','score','germlineOrSomatic','oneOrZero']
GENECNV_COLUMNS = ['patientName','gene','chrom','posStart','posEnd','minCopyNum','cnvLabel']

In [ ]:
# LOAD BED
BED_PATH = "/Users/peterpriestley/hmf/analyses/giabTruthsets/"
BED_FILE_NAME = "NA12878_GIAB_highconf_IllFB-IllGATKHC-CG-Ion-Solid_ALLCHROM_v3.2.2_highconf.bed"
#BED_PATH = "/Users/peterpriestley/hmf/analyses/slices/"  #HMF_Slicing.bed
#BED_FILE_NAME = "HMF_Slicing.bed" 
USE_BED = True
LOAD_FULL_FILE = True

### FUNCTIONS

In [ ]:
def findFiles(path,suffix):
    files=[]
    for x in os.listdir(path):
        if x[-len(suffix):] == suffix:
            files.append(x)
    return files

In [ ]:
def loadCNVforPatient(cnvFile,patientName):
    dfTumorCNV = pd.read_table(cnvFile, names = CNV_COLUMNS )
    #add regions with default copy number
    last = dfTumorCNV.iloc[0]
    for i in range(1, dfTumorCNV.shape[0]-1):
        if last['posEnd']<>dfTumorCNV.iloc[i]['posStart']:
            if last['chrom']==dfTumorCNV.iloc[i]['chrom']:
                dfTumorCNV.loc[len(dfTumorCNV)] = [last['chrom'],last['posEnd'], dfTumorCNV.iloc[i]['posStart']-1,2,'none','','',0,0]
            else:
                dfTumorCNV.loc[len(dfTumorCNV)] = [last['chrom'],last['posEnd'], cd.chromosomeLength[last['chrom']],2,'none','','',0,0]
                if dfTumorCNV.iloc[i]['posStart']<>0:
                    dfTumorCNV.loc[len(dfTumorCNV)] = [dfTumorCNV.iloc[i]['chrom'],0, dfTumorCNV.iloc[i]['posStart'],2,'none','','',0,0]
        last = dfTumorCNV.iloc[i]
    #fix first record
    if dfTumorCNV.iloc[0]['posStart']<>0:
        dfTumorCNV.loc[len(dfTumorCNV)] = ['1',0, dfTumorCNV.iloc[0]['posStart'],2,'none','','',0,0]
    #Additional Fields
    dfTumorCNV['chromPos']= dfTumorCNV['chrom'].apply(lambda x: cd.intChrom(x)) + dfTumorCNV['posStart'] / dfTumorCNV['chrom'].apply(lambda x: cd.chromosomeLength[str(x)])
    dfTumorCNV['chrom'] = dfTumorCNV['chrom'].apply(lambda x: cd.intChrom(x))
    dfTumorCNV['cappedCopyNum'] = dfTumorCNV['copyNum'].clip(upper=5)
    dfTumorCNV = dfTumorCNV.sort_values(["chrom","posStart","posEnd"]).reset_index()
    dfTumorCNV['regionLength'] = (dfTumorCNV['posEnd']-dfTumorCNV['posStart'])
    dfTumorCNV['patientName']=patientName
    return dfTumorCNV

In [ ]:
def calculateMBAFBetween(chrom,minPos,maxPos):
    dftemp = dfBAF[(dfBAF.Chromosome==chrom)&(dfBAF.Position>minPos)&(dfBAF.Position<maxPos)]
    return dftemp['mBAF'].median(),dftemp['mBAF'].count()

In [ ]:
def loadBAFData(bafFile):
    df = pd.read_table(bafFile)
    return df

In [ ]:
def cnvChart(dfTumorCNV,filename,minChrom=1,maxChrom=23):
    plt.figure(figsize=[18,3])
    plt.title(filename)
    ax = plt.gca()
    ax.plot(dfTumorCNV['chromPos'], dfTumorCNV['copyNum'],drawstyle="steps-post")
    ax.axis([minChrom, maxChrom, 0, 6])
       

In [ ]:
def cnvLabel(copyNumber):
    if copyNumber < 2:
        return 'loss'
    elif copyNumber > 2:
        return 'amplification'
    else:
        return'normal'

In [ ]:
def createBAF(dfGermline):
    dfBAF = dfGermline[(dfGermline['variantType']=="SNP")&(dfGermline['normalSGT']=="0/1")&(dfGermline['normalAF']>0.4) \
                   &(dfGermline['normalAF']<0.65)&(dfGermline['normalDP']>50)&\
                   (dfGermline['normalDP']<100)][['chrom','pos','tumorAF']]
    dfBAF.columns = ['Chromosome', 'Position','BAF']
    dfBAF['Position'] = pd.to_numeric(dfBAF['Position'])
    dfBAF['mBAF']= 0.5+abs(dfBAF['BAF']-0.5)
    dfBAF['chromPos']= dfBAF['Chromosome'].apply(lambda x: cd.intChrom(x)) + dfBAF['Position'] / dfBAF['Chromosome'].apply(lambda x: cd.chromosomeLength[str(x)])
    dfBAF['Chromosome']= dfBAF['Chromosome'].apply(lambda x: cd.intChrom(x))
    return dfBAF

In [ ]:
def patientIDFromFilename(filename):
    patientIDStart = filename.find("CPCT02")
    return filename[patientIDStart:patientIDStart+12]

In [ ]:
def germlineVCFSampleNames(filename,patientID):
    return {patientID+'R':'normal',patientID+'T':'tumor'}

In [ ]:
def loadVCF(path,filename,sampleNames,patientID):
    df = pd.DataFrame()
    if USE_BED:
        bed = aVCF.loadBEDFile(BED_PATH,BED_FILE_NAME)
        return pd.concat([df,aVCF.loadVariantsFromVCF(path,filename,sampleNames,patientID,True,True,bed,LOAD_FULL_FILE)])
    else:
        return pd.concat([df,aVCF.loadVariantsFromVCF(path,filename,sampleNames,patientID,True)])

In [ ]:
def AFByPosPlot(pos,AF,startChartPos,endChartPos):
    plt.figure(figsize=[18,3])
    plt.scatter(pos, AF)
    plt.grid(b=True, which='both', color='0.65',linestyle='-')
    plt.axis([startChartPos, endChartPos,0, 1.0])
    plt.show()

In [ ]:
def loadPON(aPath,aPONFile):
    myPON = []
    with open(aPath + aPONFile, 'r') as f:
        for line in f:
            line = line.strip('\n')
            splitLine = line.split('\t')
            myPON.append(splitLine)
    dfPON = pd.DataFrame(myPON)
    dfPON.columns = ['chrom','pos','ref','alt','PONCount']
    return dfPON

In [ ]:
def positionChart(dfCNV,filename,fieldName,minChrom=1,maxChrom=23,maxValue=10):
    plt.figure(figsize=[18,2])
    plt.title(filename+" "+fieldName)
    ax = plt.gca()
    ax.plot(dfCNV['chromPos'], dfCNV[fieldName],drawstyle="steps-post")
    ax.axis([minChrom, maxChrom, 0, maxValue])

In [ ]:
def findPeaks(log_dens,X_range):
    peaks = []
    troughs = []
    dens=np.exp(log_dens)
    diff = [dens[x] - dens[x-1] for x in range(1,len(dens))]
    for i in range (len(diff)-1):
        if diff[i+1] < 0 and diff[i] > 0:
            peaks.append(float(i+1)/len(dens)*X_range)
        if diff[i+1] > 0 and diff[i] < 0:
            troughs.append(float(i+1)/len(dens)*X_range)
    return peaks#, troughs

In [ ]:
def nonZeroCountBetween(chrom,minPos,maxPos):
    temp = dfTumorRatio[(dfTumorRatio.Chromosome==chrom)&(dfTumorRatio.Start>minPos)&\
                       (dfTumorRatio.Start<maxPos)&(dfTumorRatio.Ratio_x<>-1)]
    return temp['Start'].count(),temp['Ratio_x'].mean(),temp['Ratio_y'].mean(),temp['MedianRatio_x'].mean(),temp['MedianRatio_y'].mean()

In [ ]:
def fitBAFPeak(mBAF):
    return min(mBAFPeaks, key=lambda x:abs(x-mBAF))

def fitCNVPeak(ratio):
    return min(cnvPeaks, key=lambda x:abs(x-ratio))

In [ ]:
def pdfChart(log_dens,maxYValue=8):
    fig, ax = plt.subplots()
    ax.plot(X_plot[:, 0], np.exp(log_dens), '-',
            label="kernel = '{0}'".format('gaussian'))
    ax.legend(loc='upper right')
    fig.set_figwidth(10)
    fig.set_figheight(2)
    #ax.set_xlim(0, 1)
    ax.set_ylim(-0.02, maxYValue)
    ax.grid(b=True, which='both', color='0.65',linestyle='-')
    plt.show()

In [ ]:
def runGeneCNVAnalysis(dfCNV,bed,patientName):
    geneList =[]
    for bedrow in bed:
        bedChromPosStart = cd.intChrom(bedrow[0]) + float(bedrow[1])/cd.chromosomeLength[bedrow[0]]
        bedChromPosEnd = cd.intChrom(bedrow[0]) + float(bedrow[2])/cd.chromosomeLength[bedrow[0]]
        prevCNV = max(dfCNV[(dfCNV['chromPos'] < bedChromPosStart)].index.tolist())
        if dfCNV[(dfCNV['chromPos'] > bedChromPosEnd) ].index.tolist():
            nextCNV = min(dfCNV[(dfCNV['chromPos'] > bedChromPosEnd) ].index.tolist())
            minCopyNum = min(([dfCNV.iloc[x]['myCopyNum']]) for x in range (prevCNV,nextCNV)) 
        else:
            minCopyNum = [-1]
        geneList.append([patientName,bedrow[3],bedrow[0],bedrow[1],bedrow[2],minCopyNum[0],cnvLabel(minCopyNum[0])])
    return geneList


### LOAD PON

In [ ]:
def loadPONvcf(PONFile):
    numHeaderRows = 0
    with open(PONFile) as fp:
        while fp.readline()[0]=='#':
            numHeaderRows = numHeaderRows+1
    dfPON = pd.read_table(PONFile,skiprows =numHeaderRows-1, dtype={'#CHROM':'str','POS':'str'})  #names = CNV_COLUMNS
    dfPON['PON_COUNT'] = dfPON['INFO'].apply(lambda x: x.split('=')[1])
    dfPON.rename(columns={'#CHROM': 'chrom', 'POS': 'pos','REF':'ref','ALT':'alt'}, inplace=True)
    return dfPON

dfPON = loadPONvcf("/Users/peterpriestley/hmf/analyses/PON/PON.vcf")

### FIND ALL FILES

In [ ]:
PATH = "/Users/peterpriestley/hmf/analyses/COLO829/"

In [ ]:
germlineVCFFilename = findFiles(PATH,"SLICED.vcf")[0]
somaticVCFFilename = findFiles(PATH,"melted.vcf")[0]
patientID = "COLO829"
tumorCNVFileName=findFiles(PATH,"gz_CNVs")[0]
normalCNVFileName=findFiles(PATH,"normal_CNVs")[0]
tumorRatioFileName=findFiles(PATH,"gz_ratio.txt")[0]
normalRatioFileName=findFiles(PATH,"gz_normal_ratio.txt")[0]
print "germline =",germlineVCFFilename
print "somatic =",somaticVCFFilename
print "patient =",patientID
print "CNV =",tumorCNVFileName

### LOAD VCFS

In [ ]:
#LOAD VCFS
dfGermline = loadVCF(PATH,germlineVCFFilename,{patientID+'BL':'normal',patientID:'tumor'},patientID)
dfSomatic = loadVCF(PATH,somaticVCFFilename,{patientID:'melted'},patientID)

In [ ]:
somaticTruthVCFFilename = findFiles(PATH,"validated.vcf")[0]

In [ ]:
#dfSomaticTruth = loadVCF(PATH,somaticTruthVCFFilename,{'COLO_829_Illumina':'truth','COLO_829_EPleasance':'EP','COLO_829_TGEN':'TGEN','COLO_829_GSC':'GSC'}
#,patientID)


In [ ]:
dfSomaticTruth = loadVCF(PATH,somaticTruthVCFFilename,{'COLO_829_Illumina':'truth'},patientID)
dfSomaticTruth.head()

In [ ]:
dfSomaticTruth['alt'] = dfSomaticTruth['truthallele']
dfSomatic['alt'] = dfSomatic['meltedallele']
dfSomaticTruth = dfSomaticTruth[['chrom','pos','ref','alt','truthDP','truthAF']]
dfSomatic = pd.merge(dfSomatic,dfSomaticTruth,how='left', on=['chrom','pos','ref','alt'])
dfSomatic['inTruth']=(~dfSomatic.truthAF.isnull())

In [ ]:
# APPLY PON to SOMATICs
dfSomatic['alt'] = dfSomatic['meltedallele']
dfSomatic = pd.merge(dfSomatic,dfPON,how='left', on=['chrom','pos','ref','alt'])
dfSomatic['PON_COUNT'].fillna(0, inplace=True)
dfSomatic['inPON'] = pd.to_numeric(dfSomatic.PON_COUNT,errors=coerce)>4

In [ ]:
pd.pivot_table(dfSomatic[dfSomatic.consensus==True], values='pos', index=['variantType'], columns=['inPON'], aggfunc='count',margins=True).fillna("")

In [ ]:
for truth in [True,False]:
    ser = dfSomatic[(dfSomatic['consensus']==True)&(dfSomatic['inPON']==False)&(dfSomatic.inTruth==truth)&(dfSomatic.variantType=='INDEL')]['meltedAF']
    ser = ser.sort_values()
    cum_dist = np.linspace(0.,1.,len(ser))
    ser_cdf = pd.Series(cum_dist, index=ser,name=str(truth)+": c="+str(ser.count())+" m="+str(round(ser.median(),2)))
    ser_cdf.plot(drawstyle='steps',legend=True,title=" Allelic Frequency for SNP in Truth Set and NOT in Truth Set",figsize=[15,3],xlim=[0,1])

In [ ]:
variantType = 'INDEL'
plt.figure(figsize=[18,4])
plt.hist([dfSomatic[(dfSomatic['consensus']==True)&(dfSomatic['inPON']==False)&(dfSomatic.inTruth==True)&
    (dfSomatic.variantType==variantType)]['meltedAF'].values,dfSomatic[(dfSomatic['consensus']==True)&
    (dfSomatic['inPON']==False)&(dfSomatic.inTruth==False)&(dfSomatic.variantType==variantType)]['meltedAF'].values],
    cumulative=False, normed=True, bins=50,range=(0.0,1.0),histtype='bar',stacked = False,rwidth=0.8,label=['True','False'])
plt.title('normalised count of variant type '+variantType+' by AF')
plt.legend()
plt.show()

### Load BAF and CNV and calculate median BAF per segment

In [ ]:
dfBAF = createBAF(dfGermline)

In [ ]:
dfBAF[['Chromosome','Position','mBAF']].to_csv(PATH+'COLO829.BAF','\t')

In [ ]:
dfBAF[['Chromosome','Position','mBAF']].head()

In [ ]:
dfTumorCNV = loadCNVforPatient(PATH+tumorCNVFileName,patientID)
dfNormalCNV = loadCNVforPatient(PATH+normalCNVFileName,patientID)
dfTumorCNV['mBAF'], dfTumorCNV['mBAFCount'] = np.vectorize(calculateMBAFBetween)(dfTumorCNV['chrom'], dfTumorCNV['posStart'],dfTumorCNV['posEnd'])
cnvChart(dfTumorCNV,tumorCNVFileName,1,24)

### TUMOR PURITY

In [ ]:
c,fractionMin,fractionMax = 1,0,22
dfSom = dfBAF[(dfBAF.chromPos>c+fractionMin)&(dfBAF.chromPos<c+fractionMax)]
AFByPosPlot(dfSom.chromPos, dfSom.BAF,c+fractionMin, c+fractionMax)
# CNV according to freeC
plt.figure(figsize=[5,2])
plt.hist(dfTumorCNV['copyNum'].values,cumulative=False, normed=True, bins=20,range=(0,10),weights=dfTumorCNV['regionLength'].values)
plt.show()
# should only use regions with mbaf > x
c,fractionMin,fractionMax = 1,0,22
dftemp = dfTumorCNV[(dfTumorCNV.mBAF>=0.0)&(dfTumorCNV.mBAF<=1.0)&(dfTumorCNV.mBAFCount>20)&(dfTumorCNV.chromPos<=c+fractionMax)&(dfTumorCNV.chromPos>=c+fractionMin)]
plt.figure(figsize=[18,2])
for CNV in range(0,6):
    plt.hist(dftemp[(dftemp.copyNum==CNV)]['mBAF'].values,cumulative=False, normed=False, bins=50,range=(0.5,1.0),weights=dftemp[(dftemp.copyNum==CNV)]['mBAFCount'].values,stacked = True,rwidth=0.8,label='CNV ='+str(CNV))
    plt.legend()
plt.show()

In [ ]:
dfTumorCNV[(dfTumorCNV.chrom==10)&(dfTumorCNV.copyNum>=0)&(dfTumorCNV.mBAF>0.56)&(dfTumorCNV.mBAF<10.62)\
      &(dfTumorCNV.mBAFCount>100)][['chrom','posStart','copyNum','chromPos','regionLength','mBAF','mBAFCount']]

### SOMATIC AF

In [ ]:
#dfSomatic[(dfSomatic.consensus==True)&((dfSomatic.annWorstImpact=='MODERATE')|(dfSomatic.annWorstImpact=='HIGH'))]

In [ ]:
pd.pivot_table(dfSomatic[dfSomatic.consensus==True], values='pos', index=['patientName','variantType','inPON'], columns=['inTruth'], aggfunc='count',margins=False).fillna("")

In [ ]:
pd.pivot_table(dfSomatic[(dfSomatic.variantType == 'SNP')&(dfSomatic.consensus == True)&(dfSomatic.inPON == False)], values='pos', index=['ref'], columns=['meltedallele'], aggfunc='count',margins=True).fillna("")

In [ ]:
c,fractionMin,fractionMax = 1,0,22
dfSom = dfSomatic[(dfSomatic['variantType']=="SNP")&(dfSomatic['inPON']==False)&(dfSomatic['consensus']==True)&(dfSomatic.chromFrac>c+fractionMin)&(dfSomatic.chromFrac<c+fractionMax)]
AFByPosPlot(dfSom.chromFrac, dfSom.meltedAF,c+fractionMin, c+fractionMax)

In [ ]:
c,fractionMin,fractionMax = 12,0,1
dfTemp = dfBAF[(dfBAF.chromPos>c+fractionMin)&(dfBAF.chromPos<c+fractionMax)]
AFByPosPlot(dfTemp.chromPos, dfTemp.BAF,c+fractionMin, c+fractionMax)
dfSom = dfSomatic[(dfSomatic['variantType']=="SNP")&(dfSomatic['inPON']==False)&(dfSomatic['consensus']==True)&(dfSomatic.chromFrac>c+fractionMin)&(dfSomatic.chromFrac<c+fractionMax)]
AFByPosPlot(dfSom.chromFrac, dfSom.meltedAF,c+fractionMin, c+fractionMax)

#CDF
ser = dfSom.meltedAF.sort_values()
cum_dist = np.linspace(0.,1.,len(ser))
ser_cdf = pd.Series(cum_dist, index=ser,name=": c="+str(ser.count())+" m="+str(round(ser.median(),2)))
ser_cdf.plot(drawstyle='steps',legend=True,title=" Allelic Frequency by ",figsize=[15,2],xlim=[0.0,0.7])

#mbaf
dfTumorCNV[(dfTumorCNV.chromPos>c+fractionMin)&(dfTumorCNV.chromPos<c+fractionMax)&(dfTumorCNV.mBAFCount>200)][['chrom','posStart','copyNum','chromPos','regionLength','mBAF','mBAFCount']]

### Somatic Peak Finding

In [ ]:
X = dfSom.meltedAF.sort_values()[:, np.newaxis]
X_plot = np.linspace(0,1,100)[:, np.newaxis]
kde = KernelDensity(kernel='gaussian', bandwidth=0.015).fit(X)
log_dens = kde.score_samples(X_plot)
pdfChart(log_dens)
findPeaks(log_dens,1)

### RATIO

In [ ]:
dfTumorRatio = pd.read_table(PATH + tumorRatioFileName, header=0)#names = ['chrom','window',"ratio","segmentRatio","copyNum","mbaf","baf","GT","uncertainty"] )
dfNormalRatio = pd.read_table(PATH + normalRatioFileName, header=0)# names = ['chrom','window',"ratio","segmentRatio","copyNum","mbaf","baf","GT","uncertainty"] )
dfTumorRatio.set_index(['Chromosome','Start'],inplace=True)
dfNormalRatio.set_index(['Chromosome','Start'],inplace=True)
dfTumorRatio = pd.merge(dfTumorRatio,dfNormalRatio,how='inner', left_index=True,right_index=True)
#dfTumorRatio = dfTumorRatio[(dfTumorRatio.Ratio_y > 0)]
dfTumorRatio['ratio']= pd.to_numeric(dfTumorRatio['Ratio_x'],errors=coerce) /pd.to_numeric(dfTumorRatio['Ratio_y'],errors=coerce)
dfTumorRatio = dfTumorRatio.reset_index()
dfTumorRatio['chromPos'] = dfTumorRatio['Chromosome'].apply(lambda x: cd.intChrom(str(x))) + \
    dfTumorRatio['Start']/dfTumorRatio['Chromosome'].map(lambda x: cd.chromosomeLength[str(x)])
dfTumorRatio['Chromosome'] = dfTumorRatio['Chromosome'].apply(lambda x: cd.intChrom(str(x)))

In [ ]:
dfTumorCNV['nonZeroCount'],dfTumorCNV['tumorRatio'],dfTumorCNV['normalRatio'] \
    ,dfTumorCNV['tumorMedianRatio'],dfTumorCNV['normalMedianRatio'] = \
    np.vectorize(nonZeroCountBetween)(dfTumorCNV['chrom'], dfTumorCNV['posStart'],dfTumorCNV['posEnd'])
dfTumorCNV['ratioOfRatio']=dfTumorCNV.tumorRatio/dfTumorCNV.normalRatio

In [ ]:
positionChart(dfTumorRatio,"ratio","ratio",10.66,10.67,6)

In [ ]:
positionChart(dfTumorCNV,"ratioOfRatio","ratioOfRatio",10.66,10.67,2)
positionChart(dfTumorCNV,"tumor Ratio","tumorRatio",10.66,1.67,6)

In [ ]:
X = dfTumorCNV[dfTumorCNV.nonZeroCount>10].ratioOfRatio.sort_values()[:, np.newaxis]
X_range = 3
X_plot = np.linspace(0,X_range,1000)[:, np.newaxis]
kde = KernelDensity(kernel='gaussian', bandwidth=0.01).fit(X)
log_dens = kde.score_samples(X_plot)
pdfChart(log_dens,6)
cnvPeaks = findPeaks(log_dens,X_range)

In [ ]:
X = dfTumorCNV[dfTumorCNV.mBAFCount>10].mBAF.sort_values()[:, np.newaxis]
X_range = 1.02
X_plot = np.linspace(0,X_range,1020)[:, np.newaxis]
kde = KernelDensity(kernel='gaussian', bandwidth=0.005).fit(X)
log_dens = kde.score_samples(X_plot)
pdfChart(log_dens,15)
mBAFPeaks = findPeaks(log_dens,X_range)

In [ ]:
dfTumorCNV['fittedBAFPeak'] = np.vectorize(fitBAFPeak)(dfTumorCNV['mBAF'])
dfTumorCNV['fittedCNVPeak'] = np.vectorize(fitCNVPeak)(dfTumorCNV['ratioOfRatio'])
pd.pivot_table(dfTumorCNV[dfTumorCNV.chrom==4], values='mBAFCount', index=['fittedBAFPeak'], columns=['fittedCNVPeak'], aggfunc='sum',margins=True).fillna("")

### GENE CNV ANALYSIS

In [ ]:
# BEST ESTIMATE OF COPY NUMBER!
dfTumorCNV["myCopyNum"] = np.round(dfTumorCNV.fittedCNVPeak/0.63*2,0)

In [ ]:
# LOAD CNV BED
CNV_BED_PATH = "/Users/peterpriestley/hmf/analyses/slices/"
CNV_BED_FILE_NAME = "HMF_Slicing.Bed"
CNVBed = aVCF.loadBEDFile(CNV_BED_PATH,CNV_BED_FILE_NAME)

In [ ]:
dfGeneCNV = pd.DataFrame(runGeneCNVAnalysis(dfTumorCNV,CNVBed,'COLO829'),columns= GENECNV_COLUMNS)

In [ ]:
(dfGeneCNV[dfGeneCNV.minCopyNum<2])

In [ ]:
dfSomatic[(dfSomatic.annWorstImpact=='HIGH')&(dfSomatic.bedRegion=='')].sort_values('annGene')

### SV Analysis

In [ ]:
svVCFFilename = findFiles(PATH,"somaticSV.vcf")[0]
print "SV =",svVCFFilename
dfSV = loadVCF(PATH,svVCFFilename,{patientID:'tumor'},patientID)

In [ ]:
pd.pivot_table(dfSV[(dfSV.variantSubType<>"IGN")&(dfSV.tumorAF>=0.00)], values='id', index=['variantSubType'], columns=['variantType'], aggfunc='count').fillna("")

In [ ]:
#dfTumorCNV[(dfTumorCNV.chrom==10)&(dfTumorCNV.copyNum>=0)&(dfTumorCNV.mBAF>0.05)&(dfTumorCNV.mBAF<10.62)\
#      &(dfTumorCNV.mBAFCount>100)][['chrom','posStart','copyNum','chromPos','regionLength','mBAF','mBAFCount','fittedCNVPeak','tumorRatio','normalRatio']]

In [ ]:
dfSV[(dfSV.bedRegion<>"A")&(dfSV.variantSubType=="BND")&(dfSV.tumorAF>=0.05)][['chrom','pos','chromFrac','variantSubType','bedRegion','tumorallele','tumorAF','tumorSGT','tumorSVLenMin','tumorSVLenMax','tumorSVStartMin','tumorSVStartMax']]

In [ ]:
def findMatchingBNDChromPos(df):
    # can 
    df['BNDMatchPos'] = ""
    for i in range(0,df.shape[0]):
        if df.iloc[i]['variantSubType'] == "BND":
            tmp = df.iloc[i]['tumorallele'].find("]")
            if tmp >= 0:
                df.iloc[i,df.columns.get_loc('BNDMatchPos')] = df.iloc[i]['tumorallele'][tmp+1:df.iloc[i]['tumorallele'].find("]",tmp+1)]
    return df

In [ ]:
dfSV = findMatchingBNDChromPos(dfSV)

In [ ]:
c,fractionMin,fractionMax = 9,0,1
dfSom = dfSomatic[(dfSomatic['variantType']=="SNP")&(dfSomatic['inPON']==False)&(dfSomatic['consensus']==True)&(dfSomatic.chromFrac>c+fractionMin)&(dfSomatic.chromFrac<c+fractionMax)]
AFByPosPlot(dfSom.chromFrac, dfSom.meltedAF,c+fractionMin, c+fractionMax)
dfTemp = dfBAF[(dfBAF.chromPos>c+fractionMin)&(dfBAF.chromPos<c+fractionMax)]
AFByPosPlot(dfTemp.chromPos, dfTemp.BAF,c+fractionMin, c+fractionMax)

In [ ]:
dfSV[(dfSV.variantSubType=="INS")&(dfSV.tumorAF>=0.08)][['chrom','pos','chromFrac','variantSubType','bedRegion','BNDMatchPos','tumorallele','tumorAF','tumorDP','tumorSGT','tumorSVLenMin','tumorSVLenMax','tumorSVStartMin','tumorSVStartMax']]

In [ ]:
### MAKE SLICING FILE
dfSV.minBedPos
dfSV = dfSV[['chrom','pos','pos']]

In [ ]:
dfSVBed = []
for i in range(0, dfSV.shape[0]-1):
    if dfSV.iloc[i]['tumorSVLenMax']>1000:
        dfSVBed.append(['chr'+dfSV.iloc[i]['chrom'],int(dfSV.iloc[i]['pos'])-200,int(dfSV.iloc[i]['pos'])+200])
        dfSVBed.append(['chr'+dfSV.iloc[i]['chrom'],int(dfSV.iloc[i]['pos'])+dfSV.iloc[i]['tumorSVLenMin']-200,int(dfSV.iloc[i]['pos'])+dfSV.iloc[i]['tumorSVLenMax']+200])
    else:
        dfSVBed.append(['chr'+dfSV.iloc[i]['chrom'],int(dfSV.iloc[i]['pos'])-200,int(dfSV.iloc[i]['pos'])+dfSV.iloc[i]['tumorSVLenMax']+200])

In [ ]:
import csv

with open(PATH+"SVSlice.bed", "wb") as f:
    writer = csv.writer(f,delimiter="\t")
    writer.writerows(dfSVBed)